In [159]:
!mkdir "../datasets"
!mkdir "../datasets/LSTM"

mkdir: cannot create directory ‘../datasets’: File exists


In [131]:
!mkdir "../TT_data"

mkdir: cannot create directory ‘../TT_data’: File exists


In [148]:

!ls '../datasets'


In [ ]:
#LATEST VERSION: ONLY INCLUDE THE WORD GROUPS THAT HAVE A GESTURE (NO NON-GESTURE CLASSIFIED WORD SEQUENCES)
#DOES NORMAL CLASSIFICTAION(NOT WEIGHTED), NO MULTIPLE LAYERS

In [1]:
# The code was removed by DSX for sharing.

0     After waiting what seemed like  G52   forever...
1     Yeah   G15   it was such a change! I  G40   s...
2       G13  And I was so glad that all of the meas...
3     Ben and I had been saying for the past two we...
4     Of course   G20  that is what happened for th...
Name: sentence, dtype: object

In [2]:

class data():
    def __init__(self):
        self.data = []
        
class mini_data():
    def __init__(self):
        self.mini_data = []
        
d = data()

'''separate words in sentences and store them in lists'''
def read_data(sentence):
    md = mini_data()
    sentence = sentence.lower()
    #content = sentence.split()
    content = re.findall(r"[\w']+|[.,!?;]", sentence)
    new = md.mini_data + content
    d.data.append(new)
    
df_data['sentence'].apply(read_data)
training_data = np.array(d.data)
print(training_data[0])

print('training_data length: ' + str(len(training_data)))

avrgs = [len(x) for x in training_data]
print('longest sentence: ' + str(np.max(avrgs)))
print('shortest sentence: ' + str(np.min(avrgs)))
print('avrg sentence length: ' + str(np.mean(avrgs)))
print('standard deviation of sentence length: ' + str(np.std(avrgs)))

'''sentence length will be the average length plus the standard deviation'''
sentence_length = round(np.mean(avrgs) + np.std(avrgs))
print('sentence length: ' + str(sentence_length))


['after', 'waiting', 'what', 'seemed', 'like', 'g52', 'forever', 'to', 'get', 'into', 'the', 'room', '.', 'we', 'g28', 'finally', 'got', 'to', 'see', 'our', 'little', 'one', 'for', 'the', 'first', 'time', 'in', '7', 'weeks', 'g15', 'g44', 'and', 'wow', 'what', 'a', 'change', 'it', 'was', '!']
training_data length: 874
longest sentence: 160
shortest sentence: 1
avrg sentence length: 29.1830663616
standard deviation of sentence length: 17.4226866681
sentence length: 47.0


In [5]:
import collections

def filter_gestures(word):
    ges = re.search(r'g\d{1,2}',word)
    if ges is None:
        return True
    else:
        return False

def build_dictionary(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)+1
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

'''make the list of list of words into list of words'''
words = [item for sublist in training_data for item in sublist]

'''filter the gesture annotations'''
words = [x for x in words if filter_gestures(x)]

'''count the unique appearance of words and assign them a number'''
dictionary, reverse_dictionary = build_dictionary(words)

#print(dictionary)
vocab_size = len(dictionary)
print(vocab_size)


3214


In [316]:
dict_df = pd.DataFrame(list(dictionary.items()))
dict_df.to_csv('../TT_data/dictionary.csv', index=False)

revdict_df = pd.DataFrame(list(reverse_dictionary.items()))
revdict_df.to_csv('../TT_data/rev_dictionary.csv', index=False)

In [6]:
import tensorflow as tf
from tensorflow.contrib import rnn

#sequence length
time_steps=4

#could be any number
num_units=10

#self explanatory
input_size=1

#we need to learn about this
learning_rate=0.001

#57 getsure types + no gesture
n_classes=58
no_gesture_index = 57

#tbd
batch_size=50


In [7]:

def create_input_output(flat_data):
    
    #initialize variables
    train_input = []
    train_output = []
    
    single_input = []
    single_output = [0]*n_classes    
    
    #'''for every word in the data set'''
    for i in range(len(flat_data)):
    
        count = 0
        extra_index = 0
        
        #'''get them in groups of four'''
        words = flat_data[i+extra_index:i+time_steps+extra_index]
        
        #'''if we have reached the end of the dataset end this function'''
        if len(words) != time_steps:
            break
        
        #'''until the single_output is full'''
        while len(single_input) < time_steps:
            
            word = words[count] 
            gesture = re.search(r'g\d{1,2}', word)

            #'''if the word being analysed is not a gesture'''
            if gesture is None:

                #'''find correspondig value in dictionary and add to input'''
                num = dictionary[word]
                single_input.append(num)
                

            #'''if the word being analysed is a gesture'''
            elif gesture is not None:
            
                
                #'''get the next word from the flatlist and add it to the word to be analysed array'''
                new_word = flat_data[i+time_steps+extra_index]
                words.append(new_word)
                extra_index = extra_index + 1
               
            
            #'''when the input array is full'''
            if len(single_input) == time_steps:
                
                gesture_check = None
                
                #'''get the next entry in the flat list and check if it's a gesture'''
                if i+time_steps+extra_index < len(flat_data):
                    next_entry = flat_data[i+time_steps+extra_index]
                    gesture_check = re.search(r'g\d{1,2}', next_entry)
                
                #'''if yes, mark that gesture in the output of this particular input'''
                if gesture_check is not None:
                    index_gesture = int(next_entry[1:])
                    single_output[index_gesture] = 1
                    
                #'''if no, mark the no gesture cell in the output array'''
                else:
                    single_output[no_gesture_index] = 1
                    
                
            count = count+1
                
            
        train_input.append(single_input)
        train_output.append(single_output)  

        single_input = []
        single_output = [0]*n_classes  
    
    return train_input, train_output
            
#'''make list from list of lists'''
flat_training_data = [item for sublist in training_data for item in sublist]

#'''populate the input and output data'''
train_input, train_output = create_input_output(flat_training_data)
                       
print(len(flat_training_data))                 
print(len(train_input))
print(len(train_output))

25506
25503
25503


In [8]:
def get_indexes_to_be_filtered(list):
    
    indices = []
    
    for i in range(0,len(list)):
        if list[i][no_gesture_index]==1:
            indices.append(i)
        
    return indices

indices = get_indexes_to_be_filtered(train_output)
train_actual_output = [train_output[i] for i in range(0, len(train_output)) if i not in indices]
len(train_actual_output)

train_actual_input = [train_input[i] for i in range(0, len(train_input)) if i not in indices]
len(train_actual_input)

    

2210

In [9]:
#creating the weight array for the weighted loss function

counts = [item.index(1) for item in train_output]
count_total = [counts.count(i) for i in range(0,58)]
print(count_total)

weight_array_1 = [1 - (item/sum(count_total)) for item in count_total]
#last_we = weight_array_1[no_gesture_index]
#weight_array_1 = [0.99]*n_classes
#weight_array_1[no_gesture_index] = last_we
#weight_array_1


[30, 59, 27, 17, 94, 55, 27, 63, 97, 95, 14, 26, 45, 40, 27, 41, 34, 30, 61, 16, 49, 71, 46, 50, 42, 101, 21, 20, 37, 34, 7, 38, 18, 35, 99, 109, 23, 52, 27, 15, 12, 13, 17, 19, 28, 21, 14, 31, 15, 25, 22, 10, 19, 24, 123, 6, 19, 23293]


In [385]:
weight_array_2 = [weight/sum(weight_array_1) for weight in weight_array_1]
weight_array_2

[0.01752322224217171,
 0.017503272748785655,
 0.01752528598286682,
 0.017532165118517185,
 0.017479195774009384,
 0.017506024403045804,
 0.01752528598286682,
 0.01750052109452551,
 0.017477132033314274,
 0.017478507860444345,
 0.017534228859212294,
 0.017525973896431856,
 0.017512903538696164,
 0.017516343106521347,
 0.01752528598286682,
 0.017515655192956312,
 0.017520470587911566,
 0.01752322224217171,
 0.017501896921655585,
 0.01753285303208222,
 0.01751015188443602,
 0.01749501778600522,
 0.01751221562513113,
 0.017509463970870984,
 0.017514967279391273,
 0.01747438037905413,
 0.01752941346425704,
 0.017530101377822075,
 0.017518406847216457,
 0.017520470587911566,
 0.01753904425416755,
 0.01751771893365142,
 0.01753147720495215,
 0.01751978267434653,
 0.0174757562061842,
 0.017468877070533836,
 0.017528037637126966,
 0.01750808814374091,
 0.01752528598286682,
 0.01753354094564726,
 0.017535604686342368,
 0.01753491677277733,
 0.017532165118517185,
 0.01753078929138711,
 0.01752459

In [15]:
#Random sampling training and test data
_TRAINING_INS_COUNT = 2100
train_indexes = random.sample(range(0, len(train_actual_input)), _TRAINING_INS_COUNT)
test_indexes = [i for i in range(0, len(train_actual_input)) if not i in train_indexes]


#creating training data 80%
_TRAINING_INPUT = np.array(train_actual_input)
_TRAINING_INPUT = _TRAINING_INPUT[train_indexes]
_TRAINING_OUTPUT = np.array(train_actual_output)
_TRAINING_OUTPUT = _TRAINING_OUTPUT[train_indexes]


#creating testing data 20%
_TESTING_INPUT = np.array(train_actual_input)
_TESTING_INPUT = _TESTING_INPUT[test_indexes]
_TESTING_OUTPUT = np.array(train_actual_output)
_TESTING_OUTPUT = _TESTING_OUTPUT[test_indexes]

len(_TESTING_INPUT)



110

In [388]:
testin = pd.DataFrame(_TESTING_INPUT)
testout = pd.DataFrame(_TESTING_OUTPUT)
testin.to_csv("../TT_data/testing_input.csv", index=False)
testout.to_csv("../TT_data/testing_output.csv", index=False)

In [389]:
!ls '../TT_data'

dictionary.csv	rev_dictionary.csv  testing_input.csv  testing_output.csv


In [11]:
tf.reset_default_graph()

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]), name="weights")
out_bias=tf.Variable(tf.random_normal([n_classes]), name="biases")

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,input_size])
print(x)
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,input_size] to "time_steps" number of [batch_size,input_size] tensors
input=tf.unstack(x ,time_steps,1)
print(input)

Tensor("Placeholder:0", shape=(?, 4, 1), dtype=float32)
[<tf.Tensor 'unstack:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack:1' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack:2' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack:3' shape=(?, 1) dtype=float32>]


In [12]:
# defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)

with tf.variable_scope("rnn", reuse=None):
    outputs,state=rnn.static_rnn(lstm_layer,input,dtype="float32")
        
    #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
    prediction=tf.matmul(outputs[-1],out_weights)+out_bias

    #loss_function
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    #model evaluationmatmul
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [19]:
#initialize variables
batch_init = 0
limit = batch_init + batch_size
init=tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init)
    iter=1
    while iter<42:
        
        batch_x = _TRAINING_INPUT[batch_init: limit]
        batch_y = _TRAINING_OUTPUT[batch_init: limit]
        batch_init = batch_init + batch_size
        limit = batch_init + batch_size

        batch_x=batch_x.reshape((batch_size,time_steps,input_size))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})
        
        raw_predictions = prediction.eval(feed_dict={x: batch_x}, session=sess).tolist()
        results = [sublist.index(max(sublist)) for sublist in raw_predictions]
        print(results)

        if iter %10==0:
            print(outputs[-1])
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1        
    
    save_path = saver.save(sess,"../datasets/LSTM_1/model.ckpt")
    print(save_path)

    #calculating test accuracy
    test_data = _TESTING_INPUT.reshape((-1, time_steps, input_size))
    test_label = _TESTING_OUTPUT
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
    
    #print("Prediction:", sess.run(prediction, feed_dict={x: test_data, y: test_label}))
    #d = prediction.eval(feed_dict={x: test_data}, session=sess)
    #print ("predictions", d)
    #print(type(d[0]))
    
    #YOU PREPARED THE DATA, YOU WERE TRYING TO MAKE THE LAST BIT WORK

[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [ ]:
!ls "../datasets/LSTM"

In [ ]:
# after this point the next two cells are downloading the prediction & actual output files
#i do not need them anymore but I am keeping them there just to make sure I have the code ready in case I need it

In [ ]:
# The code was removed by DSX for sharing.

In [ ]:
# The code was removed by DSX for sharing.